<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.1 
# *Data Wrangling and Munging with Pandas*

## Part 1: Wrangling Data

The term "data wrangling" is analogous to capturing wild horses and getting them into a fenced area; the horses are data and the fencing is your computer. The more common data wrangling tasks include:

- reading flat files
- reading Excel files
- downloading from web pages
  - csv
  - html
  - json

In [4]:
import numpy as np
import pandas as pd

*It is good practice to display the library version numbers for future reference:*

In [3]:
print('Numpy: ', np.__version__)
print('Pandas: ', pd.__version__)

Numpy:  1.19.2
Pandas:  1.1.3


### CSV Files

Below are three attempts to load the file "bikeshare.csv" into a DataFrame named `bikes`. Why are they wrong?

In [5]:
#CSV is a comma delimited file, but the syntax did not split it up to individual columns
# wrong:
bikes = pd.read_table('bikeshare.csv', header = None)
print(bikes.head())
#The header became a record


# wrong:
bikes = pd.read_table('bikeshare.csv', header = 1)
print(bikes.head())
#specifying header = 1, will take the first row of data as column header 

# # wrong:
#Taking only first line of data and 
bikes = pd.read_table('bikeshare.csv', header = 0)
print(bikes.head())
#This should be correct 

                                                   0
0  instant,dteday,season,yr,mnth,hr,holiday,weekd...
1  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...
2  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...
3  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...
4  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...
  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0,3,13,16
0  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...     
1  3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.8,0...     
2  4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,...     
3  5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,...     
4  6,2011-01-01,1,0,1,5,0,6,0,2,0.24,0.2576,0.75,...     
  instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0  1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,...                                                                   
1  2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.8,0...                                          

In [4]:
help(pd.read_table)

Help on function read_table in module pandas.io.parsers:

read_table(filepath_or_buffer: Union[str, pathlib.Path, IO[~AnyStr]], sep='\t', delimiter=None, header='infer', names=None, index_col=None, usecols=None, squeeze=False, prefix=None, mangle_dupe_cols=True, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, skip_blank_lines=True, parse_dates=False, infer_datetime_format=False, keep_date_col=False, date_parser=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal: str = '.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, dialect=None, error_bad_lines=True, warn_bad_lines=True, delim_whitespace=False, low_memory=True, memory_map=False, float_precision=None)
    Read general delimited file into DataF

?:
ANSWER: Case 1 treats headings as just another data row. Case 2 treats the 1st data row as the column header. Case 3 gets the header right (row 0), but reads each row as a single column (Nb. the other two make that same mistake). 

Load the file "bikeshare.csv" into a DataFrame named `bikes`, and confirm that it was loaded properly:

In [18]:
#ANSWER:
bikes = pd.read_csv('bikeshare.csv')
bikes.head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,2011-01-01,1,0,1,0,0,6,0,1,0.24,0.2879,0.81,0.0,3,13,16
1,2,2011-01-01,1,0,1,1,0,6,0,1,0.22,0.2727,0.80,0.0,8,32,40
2,3,2011-01-01,1,0,1,2,0,6,0,1,0.22,0.2727,0.80,0.0,5,27,32
3,4,2011-01-01,1,0,1,3,0,6,0,1,0.24,0.2879,0.75,0.0,3,10,13
4,5,2011-01-01,1,0,1,4,0,6,0,1,0.24,0.2879,0.75,0.0,0,1,1


Note that we could have used `read.csv()` above. When is `read_table()` necessary?

?:
ANSWER: When `sep` is not the comma character, or we need fine control that `read.csv()` does not provide.

Flat files can be full of surprises. Here are some issues to watch out for:

- separator character is something other than the comma
  - ";", "|", and tab are popular
- newline character is something other than what the O/S expects 
  - Tip: Don't hard-code the character codes for carriage returns, linefeeds, etc. Use Python's built-in representation instead (e.g. Python translates "\n" to the newline character and "\t" to the tab character on any O/S).
- truncated lines
  - if there are empty fields at the end of a line it is possible that their separators will be missing, resulting in a "jagged" file
- embedded commas or quotes
  - a free-text field containing embedded commas may split into separate fields on input
  - a free-text field containing embedded quotes may not parse correctly
- unescaped characters
  - the "\" character indicates a control code to Python, which will break the I/O
    - e.g. the substring "\u0123" will be interpreted as Unicode(0123) -- which may not be what the file creator intended
  - these may need to be fixed by loading whole strings and then parsing into a new data frame
  
Tip: Most issues can be delth with by correctly specifying the parameters of the function you use to load the file. Read the doco before reading the data!

### Reading Excel Files

In [11]:
from pandas import ExcelFile  # Nb. Need to install xlrd from conda (it does not automatically install with pandas)

In [62]:
df = pd.read_excel('Iris.xls', sheet_name = 'Data')
df

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length,Species_name
0,1,0.2,1.4,3.5,5.1,Setosa
1,1,0.2,1.4,3.0,4.9,Setosa
2,1,0.2,1.3,3.2,4.7,Setosa
3,1,0.2,1.5,3.1,4.6,Setosa
4,1,0.2,1.4,3.6,5.0,Setosa
...,...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7,Verginica
146,3,1.9,5.0,2.5,6.3,Verginica
147,3,2.0,5.2,3.0,6.5,Verginica
148,3,2.3,5.4,3.4,6.2,Verginica


So, this file appears to have an embedded table of aggregates on the same sheet as the raw data (a naughty but common practice amongst analysts).

It is usually better to load data correctly than to meddle with the source file or load it 'warts and all' and then try to parse it in code. The Pandas functions for reading files have parameters that provide the control we need. For ecxample, we could make multiple calls to `read_excel()`, using combinations of the `header`, `usecols`, `skiprows`, `nrows`, and `skipfooter` parameters to load one table at a time from a spreadsheet with multiple tables.

Load the above file without the unwanted columns:

In [23]:
#ANSWER
df1 = pd.read_excel('Iris.xls', sheet_name = 'Data', usecols=['Petal_width','Petal_length','Sepal_width','Sepal_length'])
df1

,Species_No,Petal_width,Petal_length,Sepal_width,Sepal_length
0,1,0.2,1.4,3.5,5.1
1,1,0.2,1.4,3.0,4.9
2,1,0.2,1.3,3.2,4.7
3,1,0.2,1.5,3.1,4.6
4,1,0.2,1.4,3.6,5.0
...,...,...,...,...,...
145,3,2.3,5.2,3.0,6.7
146,3,1.9,5.0,2.5,6.3
147,3,2.0,5.2,3.0,6.5
148,3,2.3,5.4,3.4,6.2


In [64]:
df1 = pd.read_excel('Iris.xls', sheet_name = 'Data', usecols=[1,2,3,4])
df1

,Petal_width,Petal_length,Sepal_width,Sepal_length
0,0.2,1.4,3.5,5.1
1,0.2,1.4,3.0,4.9
2,0.2,1.3,3.2,4.7
3,0.2,1.5,3.1,4.6
4,0.2,1.4,3.6,5.0
...,...,...,...,...
145,2.3,5.2,3.0,6.7
146,1.9,5.0,2.5,6.3
147,2.0,5.2,3.0,6.5
148,2.3,5.4,3.4,6.2


### Importing Data Directly from the Web

We usually want to store a local copy of a data file that we download from the Web, but when data retention is not a priority it is convenient to download the data directly into our running Python environment.

#### Importing Text Files from the Web

The web is the 'wild west' of data formats. However, we can usually expect good behaviour from files that are automatically generated by a service, such as the earthquake report:

In [24]:
df2 = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/2.5_hour.csv')
df2.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource


#### Importing HTML Files from the Web

Working with unstructured HTML files relies heavily on library functions. This one, however, is well-structured:

In [25]:
url = 'https://www.fdic.gov/resources/resolutions/bank-failures/failed-bank-list/'

df3 = pd.read_html(url)
df3

[                         Bank NameBank           CityCity StateSt  CertCert  \
 0                    Almena State Bank             Almena      KS     15426   
 1           First City Bank of Florida  Fort Walton Beach      FL     16748   
 2                 The First State Bank      Barboursville      WV     14361   
 3                   Ericson State Bank            Ericson      NE     18265   
 4     City National Bank of New Jersey             Newark      NJ     21111   
 ..                                 ...                ...     ...       ...   
 558                 Superior Bank, FSB           Hinsdale      IL     32646   
 559                Malta National Bank              Malta      OH      6629   
 560    First Alliance Bank & Trust Co.         Manchester      NH     34264   
 561  National State Bank of Metropolis         Metropolis      IL      3815   
 562                   Bank of Honolulu           Honolulu      HI     21029   
 
                  Acquiring Institutio

#### Importing XML Files from the Web

XML files are semi-structured, but you're at the mercy of the file creator. If every record has the same format it will be much easier, but practical applications often require a lot of custom code. Here is an example that includes a nice parser class: http://www.austintaylor.io/lxml/python/pandas/xml/dataframe/2016/07/08/convert-xml-to-pandas-dataframe/

#### Importing JSON Files from the Web

Like XML, JSON files are semi-structured and may require work to capture the schema into a dataframe. Here is a simple example: 

In [26]:
url = 'https://raw.githubusercontent.com/chrisalbon/simulated_datasets/master/data.json'

# Load the first sheet of the JSON file into a data frame
df4 = pd.read_json(url, orient = 'columns')
df4.head()

,integer,datetime,category
0,5,2015-01-01 00:00:00,0
1,5,2015-01-01 00:00:01,0
2,9,2015-01-01 00:00:02,0
3,6,2015-01-01 00:00:03,0
4,6,2015-01-01 00:00:04,0


## Part 2: Data Munging

Data munging is manipulating data to get it into a form that we can start running analyses on (which usually means getting the data into a DataFrame). Before we get to this stage, we may need to remove headers or footers, transpose columns to rows, split wide data tables into long ones, and so on. (Nb. Excel files can be particularly troublesome, because users can format their data in mixed, complex shapes.) Essentially, we need to follow Hadley Wickham's guidelines for tidy datasets (http://vita.had.co.nz/papers/tidy-data.html):

The end goal of the cleaning data process:

- each variable should be in one column
- each observation should comprise one row
- each type of observational unit should form one table
- include key columns for linking multiple tables
- the top row contains (sensible) variable names
- in general, save data as one file per table


### Dataset Morphology

Once we have our dataset in a DataFrame (or Series, if our data is only 1-dimensional), we can start examining its size and content.

How many rows and columns are in `bikes`?

In [5]:
#ANSWER
bikes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17379 entries, 0 to 17378
Data columns (total 17 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   instant     17379 non-null  int64  
 1   dteday      17379 non-null  object 
 2   season      17379 non-null  int64  
 3   yr          17379 non-null  int64  
 4   mnth        17379 non-null  int64  
 5   hr          17379 non-null  int64  
 6   holiday     17379 non-null  int64  
 7   weekday     17379 non-null  int64  
 8   workingday  17379 non-null  int64  
 9   weathersit  17379 non-null  int64  
 10  temp        17379 non-null  float64
 11  atemp       17379 non-null  float64
 12  hum         17379 non-null  float64
 13  windspeed   17379 non-null  float64
 14  casual      17379 non-null  int64  
 15  registered  17379 non-null  int64  
 16  cnt         17379 non-null  int64  
dtypes: float64(4), int64(12), object(1)
memory usage: 2.3+ MB


What are the column names in `bikes`?

In [30]:
#ANSWER
list(bikes.columns)

['instant',
 'dteday',
 'season',
 'yr',
 'mnth',
 'hr',
 'holiday',
 'weekday',
 'workingday',
 'weathersit',
 'temp',
 'atemp',
 'hum',
 'windspeed',
 'casual',
 'registered',
 'cnt']

What are the data types of these columns?

In [31]:
#ANSWER
bikes.dtypes

instant         int64
dteday         object
season          int64
yr              int64
mnth            int64
hr              int64
holiday         int64
weekday         int64
workingday      int64
weathersit      int64
temp          float64
atemp         float64
hum           float64
windspeed     float64
casual          int64
registered      int64
cnt             int64
dtype: object

What is the (row) index for this DataFrame?

In [33]:
#ANSWER
print(bikes.index)

RangeIndex(start=0, stop=17379, step=1)


https://www.dataquest.io/blog/python-json-tutorial/

## Slicing and Dicing

It is often preferable to refer to DataFrame columns by name, but there is more than one way to do this. 
Do `bikes['season']` and `bikes[['season']]` give the same object? Demonstrate:

In [7]:
#ANSWER
type(bikes['season'])
#with one square brackets, it returns a single column series
#a series can only return on column

pandas.core.series.Series

In [55]:
type(bikes[['season','atemp']])
#with one square brackets, it returns a dataframe
#dataframe can return multiple columns

pandas.core.frame.DataFrame

How would we use object notation to show the first 4 rows of `atemp`?

In [57]:
#ANSWER
bikes.iloc[0:4,11]

0    0.2879
1    0.2727
2    0.2727
3    0.2879
Name: atemp, dtype: float64

Algorithms that loop over multiple columns often access DataFrame columns by index. However, none of the following work (try them out by uncommenting / removing the "#E: " ): 

In [ ]:
#bikes[[0]]
#bikes[0]
#bikes[0,0]
bikes[[0,0]]

What is the correct way to access the 1st row of the DataFrame by its index?

In [62]:
#ANSWER
bikes.iloc[0,:]

instant                1
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     0
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.24
atemp             0.2879
hum                 0.81
windspeed              0
casual                 3
registered            13
cnt                   16
Name: 0, dtype: object

What is the correct way to access the 2nd column of the DataFrame by its index?

In [63]:
#ANSWER
bikes.iloc[1,:]

instant                2
dteday        2011-01-01
season                 1
yr                     0
mnth                   1
hr                     1
holiday                0
weekday                6
workingday             0
weathersit             1
temp                0.22
atemp             0.2727
hum                  0.8
windspeed              0
casual                 8
registered            32
cnt                   40
Name: 1, dtype: object

## Handling Missing Values

What is the Pandas `isnull` function for? 

ANSWER: find null values in array-like objects

We can apply `isnull` to the `bikes` DataFrame to show the result for every element:

In [34]:
bikes.isnull().head()

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


However, we usually start at a higher level. How many nulls are in `bikes` altogether?

In [40]:
#ANSWER
bikes.isnull().sum().sum()

0

If this result were nonzero we would next want to find out which columns contained nulls. How can this be done in one line of code?

In [0]:
#ANSWER
bikes.isnull().sum()

What is the Numpy object `nan` used for? (Write a descriptive answer.)

Write (and verify) a function that performs scalar division with built-in handling of the edge case (i.e. return a value instead of just trapping the error):

In [14]:
#ANSWER
def divide(numerator,denominator): 
    try:
        result = numerator / denominator
    except ZeroDivisionError:
        result = 0
    except TypeError: 
        print('Numerator and Denominator needs to be numeric')
        result = None
    return result 

In [15]:
divide(1,'a')

Numerator and Denominator needs to be numeric


Apply the Pandas `isna` function to the following data objects:

In [41]:
x = 2.3
y = np.nan
print(x, y)

2.3 nan


In [45]:
#ANSWER
print(pd.isna(x))
print(pd.isna(y))

False
True


In [47]:
array = np.array([[1, np.nan, 3], [4, 5, np.nan]])
print(array)

[[ 1. nan  3.]
 [ 4.  5. nan]]


In [48]:
#ANSWER
print(pd.isna(array))

[[False  True False]
 [False False  True]]


How is the pandas I/O parameter `na_values` used?

? ANSWER: 

## Data Profiling

### Counts

When there are categorical variables in a dataset we will want to know how many possible values there are in each column. (Nb. If the dataset is a sample of a larger one, our sample may not capture all possible values of every categorical.)

How many (different) seasons are in `bikes`?

In [49]:
#ANSWER
bikes['season'].unique()

array([1, 2, 3, 4], dtype=int64)

### Ranges

Print the range of the `instant`, `dteday`, and `windspeed` columns: 

In [82]:
#ANSWER
print('range for instant is ', bikes['instant'].max()-bikes['instant'].min())
print('range for dteday is ', pd.to_datetime(bikes["dteday"]).max()-pd.to_datetime(bikes["dteday"]).min())
print('range for windspeed is ', bikes['windspeed'].max()-bikes['windspeed'].min())

range for instant is  17378
range for dteday is  730 days 00:00:00
range for windspeed is  0.8507


Compute and print the overall minimum and maximum of the numeric data columns:

In [13]:
#bikes_min, bikes_max = (min(bikes.min()), max(bikes.max()))
bikes_min, bikes_max = (bikes.min(), bikes.max())
bikes_min, bikes_max

(instant                1
 dteday        2011-01-01
 season                 1
 yr                     0
 mnth                   1
 hr                     0
 holiday                0
 weekday                0
 workingday             0
 weathersit             1
 temp                0.02
 atemp                  0
 hum                    0
 windspeed              0
 casual                 0
 registered             0
 cnt                    1
 dtype: object,
 instant            17379
 dteday        2012-12-31
 season                 4
 yr                     1
 mnth                  12
 hr                    23
 holiday                1
 weekday                6
 workingday             1
 weathersit             4
 temp                   1
 atemp                  1
 hum                    1
 windspeed         0.8507
 casual               367
 registered           886
 cnt                  977
 dtype: object)

### Quantiles

Pandas makes computing quantiles easy. This is how to get the median of a Series:

In [17]:
bikes['atemp'].quantile(0.5)

0.4848

Of course, the `quantiles` method can take a tuple as its argument. Compute the 10th, 25th, 50th, 75th, and 90th percentiles in one line of code: 

In [16]:
#ANSWER
bikes['atemp'].quantile([0.1,0.25,0.5,0.75,0.9])

0.10    0.2424
0.25    0.3333
0.50    0.4848
0.75    0.6212
0.90    0.6970
Name: atemp, dtype: float64

### Cuts

Sometimes we want to split the sample not by the quantiles of the distribution but by the range of the data. Let's take a closer look at `atemp`:

In [18]:
type(bikes['atemp'])

pandas.core.series.Series

In [19]:
bikes.sample(5)

,instant,dteday,season,yr,mnth,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
9660,9661,2012-02-12,1,1,2,11,0,0,0,1,0.12,0.1061,0.42,0.2985,3,121,124
6515,6516,2011-10-04,4,0,10,1,0,2,1,2,0.40,0.4091,0.87,0.1045,1,8,9
11117,11118,2012-04-13,2,1,4,10,0,5,1,1,0.44,0.4394,0.38,0.1940,59,133,192
12330,12331,2012-06-02,2,1,6,23,0,6,0,1,0.56,0.5303,0.52,0.1045,53,185,238
12314,12315,2012-06-02,2,1,6,7,0,6,0,1,0.52,0.5000,0.72,0.1045,9,77,86


Suppose we decide to sort these values into 4 bins of equal width, but we want to apply the resulting groups to the entire DataFrame. Basically, we need to add a row label that indcates which bin each sample belongs in. Let's call this label "atemp_level", and use the `cut` method to populate it:

In [3]:
atemp_level = pd.cut(bikes['atemp'], bins = 4) 
#pd.cut is used to segment data into bins

What is `atemp_level`?

In [26]:
#ANSWER 
type(atemp_level)
#It is a series of data, the defines the bin that each record in the bikes dataframe belongs to. 

pandas.core.series.Series

Here is a random sample of `atemp_level`:

In [7]:
atemp_level.sample(5)          

127      (0.25, 0.5]
8760     (0.25, 0.5]
16835    (0.25, 0.5]
9785     (0.25, 0.5]
4667     (0.5, 0.75]
Name: atemp, dtype: category
Categories (4, interval[float64]): [(-0.001, 0.25] < (0.25, 0.5] < (0.5, 0.75] < (0.75, 1.0]]

So, by default, `cut` produces labels that indicate the bin boundaries for each element in the series it was applied to. Usually, we will specify labels that are appropriate to the discretisation we are applying:

In [7]:
atemp_level = pd.cut(bikes['atemp'], bins = 4, labels = ["cool", "mild", "warm", "hot"])
atemp_level.sample(5)          

11258    mild
14460    warm
9020     cool
5745     warm
16578    mild
Name: atemp, dtype: category
Categories (4, object): ['cool' < 'mild' < 'warm' < 'hot']

Incorporate the new `atemp_level` column into the `bikes` DataFrame and use it to count the number of "mild" `atemp` entries in `season` 2:

In [13]:
#ANSWER
#bikes.append(atemp_level)
bikes['atemp_level'] = atemp_level
len(bikes[(bikes['season']==2) & (bikes['atemp_level']=='mild')])

1829

*Nb. The `atemp_level` variable we created is what the R language calls a "factor". Pandas has introduced a new data type called "category" that is similar to R's factors.*

# Synthetic Data

Sometimes we may want to generate test data, or we may need to initalise a series, matrix, or data frame for input to an algorithm. Numpy has several methods we can use for this.

Execute the following, then check the shape and content of each variable:

In [14]:
# Creating arrays with initial values
a = np.zeros((3))
b = np.ones((1,3))
c = np.random.randint(1,10,(2,3,4))   # randint(low, high, size)
d = np.arange(4)
e = np.array( [[1,2,3,4], [5,6,7,8]] )

In [22]:
# Cleaning Data
type(c)

numpy.ndarray

## Load Data

Load rock.csv and clean the dataset.

In [44]:
rock = pd.read_csv('rock.csv')

## Check Column Names

Check column names and clean.

In [45]:
rock.columns = [i.replace(' ', '_') for i in rock.columns.str.replace('?','').str.replace('*','').str.lower()]
rock.head()

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
0,Caught Up in You,.38 Special,1982,Caught Up in You by .38 Special,1,1,82,82
1,Fantasy Girl,.38 Special,NaN,Fantasy Girl by .38 Special,1,0,3,0
2,Hold On Loosely,.38 Special,1981,Hold On Loosely by .38 Special,1,1,85,85
3,Rockin' Into the Night,.38 Special,1980,Rockin' Into the Night by .38 Special,1,1,18,18
4,Art For Arts Sake,10cc,1975,Art For Arts Sake by 10cc,1,1,1,1


## Replace Null Values With 0

Check 'release' column whether this column have any null value or not. Replace null value with 0.

In [46]:
print(rock['release_year'].isnull().sum())
rock['release_year'].fillna('0', inplace = True)


577


In [47]:
print(rock['release_year'].isnull().sum())

0


## Check Datatypes of Dataset

Check datatypes of the dataset. Is there any column which should be int instead of object? Fix the column. 

In [48]:
rock.dtypes
rock[rock['release_year']=='SONGFACTS.COM']

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
1504,Bullfrog Blues,Rory Gallagher,SONGFACTS.COM,Bullfrog Blues by Rory Gallagher,1,1,1,1


In [49]:
#update to release year to 1972
rock.at[1504,'release_year']=1972

In [50]:
rock[rock['song_clean']=='Bullfrog Blues']

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg
1504,Bullfrog Blues,Rory Gallagher,1972,Bullfrog Blues by Rory Gallagher,1,1,1,1


In [51]:
rock['release_year'] = rock['release_year'].astype('int64')

In [52]:
rock.dtypes

song_clean      object
artist_clean    object
release_year     int64
combined        object
first            int64
year             int64
playcount        int64
fg               int64
dtype: object

## Check Min, Max of Each Column

Is there any illogical value in any column? How can we fix that?

In [77]:
rock.describe().loc[['min','max']]
#release year shouldn't be zero

,release_year,first,year,playcount,fg
min,0.0,1.0,0.0,0.0,0.0
max,2014.0,1.0,1.0,142.0,142.0


In [32]:
rock.describe()

,release_year,first,year,playcount,fg
count,2230.000000,2230.0,2230.000000,2230.000000,2230.000000
mean,1466.215695,1.0,0.741256,16.872646,15.048430
std,866.706564,0.0,0.438043,25.302972,25.288366
min,0.000000,1.0,0.000000,0.000000,0.000000
25%,0.000000,1.0,0.000000,1.000000,0.000000
50%,1973.000000,1.0,1.000000,4.000000,3.000000
75%,1981.000000,1.0,1.000000,21.000000,18.000000
max,2014.000000,1.0,1.000000,142.000000,142.000000


In [14]:
(rock['release_year']==0).sum()

577

# Write Some Functions

In [15]:
#clean up zero value with median of data set 
if (rock['release_year']==0).sum() > 0:
    rock['release_year'] = rock['release_year'].replace(0,rock['release_year'].median())

In [32]:
(rock['release_year']==0).sum()

0

## Write a function that will take a row of a DataFrame and print out the song, artist, and whether or not the release date is < 1970

In [138]:
def check_a_record():
    max_row = len(rock)
    max_p1 = max_row + 1
    value = int(input("Please enter an integer that is smaller than " + str(max_p1) + ":"))
    if value <= max_row:
        one_row = pd.DataFrame(rock.iloc[[value]])
        one_row['before_1970'] = one_row['release_year'].apply(lambda x: True if x < 1970 else False)
    else: 
        print('Please enter value smaller than ' + str(max_p1) + '.')
    return one_row.iloc[[0],[0,1,8]]

In [139]:
check_a_record()

Please enter an integer that is smaller than 2231:6


,song_clean,artist_clean,before_1970
6,Loser,3 Doors Down,False


## Write a function that converts cells in a DataFrame to float and otherwise replaces them with np.nan

In [29]:
#rock2 = pd.to_numeric(rock.value, errors='coerce')
def float_coverter(dataframe_name): 
    dataframe_name = dataframe_name.apply(pd.to_numeric, errors='coerce', downcast='float')
    return dataframe_name

## Apply these functions to your dataset

In [31]:
float_coverter(rock)

,song_clean,artist_clean,release_year,combined,first,year,playcount,fg,post_70s
0,NaN,NaN,1982.0,NaN,1.0,1.0,82.0,82.0,NaN
1,NaN,NaN,1973.0,NaN,1.0,0.0,3.0,0.0,NaN
2,NaN,NaN,1981.0,NaN,1.0,1.0,85.0,85.0,NaN
3,NaN,NaN,1980.0,NaN,1.0,1.0,18.0,18.0,NaN
4,NaN,NaN,1975.0,NaN,1.0,1.0,1.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...
2225,NaN,NaN,1973.0,NaN,1.0,0.0,1.0,0.0,NaN
2226,NaN,NaN,1981.0,NaN,1.0,1.0,32.0,32.0,NaN
2227,NaN,NaN,1975.0,NaN,1.0,1.0,109.0,109.0,NaN
2228,NaN,NaN,1983.0,NaN,1.0,1.0,1.0,1.0,NaN


## Describe the new float-only DataFrame.

In [ ]:
#The new dataframe has made columns like song_clean, artist_clean, combined redundant. We could still do analysis on
#this set of data but we will know which artist/song is more popular. 

>




---



---



> > > > > > > > > © 2021 Institute of Data


---



---



